In [3]:
!python -m cupyx.tools.install_library --cuda 11.x --library cutensor

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/cupyx/tools/install_library.py", line 303, in <module>
    main(sys.argv[1:])
  File "/usr/local/lib/python3.10/dist-packages/cupyx/tools/install_library.py", line 295, in main
    install_lib(params.cuda, params.prefix, params.library)
  File "/usr/local/lib/python3.10/dist-packages/cupyx/tools/install_library.py", line 196, in install_lib
    raise RuntimeError('''
RuntimeError: 
The destination directory /root/.cupy/cuda_lib/11.x/cutensor/1.6.2 already exists.
Remove the directory first if you want to reinstall.


In [5]:
pip install cupy-cuda12x cutensor-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 6.4 MB/s eta 0:00:00


In [17]:
import numpy as np
import cupy as cp
import cutensor as cutensor

cp.cuda.Stream.null.synchronize()

In [ ]:
arr = cp.ones((1000,500,500))

#makes us wait for gpu to finish before returning
cp.cuda.Stream.null.synchronize()

In [15]:
A = cp.array([[[1,2,3],
               [3,4,5],
               [6,7,8]],

              [[9,10,11],
               [12,13,14],
               [15,16,17]]],)

B = cp.zeros((2,3,3))

print("A: ", A)
print("B: ", B)


A:  [[[ 1  2  3]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]]
B:  [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [ ]:
result = cp.einsum('ijk,jkl->il', A, B)
print("Result with cp.einsum: ", result)

# ijk the dimensions of A, jkl the dimensions of B, il the dimensions of the einstein sum
# contraction happens over the j and k dimensions
cp.cuda.Stream.null.synchronize()

In [18]:
# using cutensor

#need descriptors to describe shape, data type and memory of tensor
desc_A = cutensor.create_tensor_descriptor(A)

desc_B = cutensor.create_tensor_descriptor(B)

# Empty output tensor of the shape of the einstein sum of dimensions il
output = cp.empty((2,3), dType = A.dtype)

desc_output = cutensor.create_tensor_descriptor(output)

AttributeError: module 'cutensor' has no attribute 'create_tensor_descriptor'

In [ ]:
#create description of contraction of the tensors
contract_desc = cutensor.einsum_expr(('i','j','k'),('j','k','l'),('i','l'))
#('i','j','k') dimensions of A,
#('j','k','l') dimensions of B,
#('i','l') dimensions of output

#scale factor for contraction operation
alpha = 1.0
#scale factor for output
beta = 0.0

#contraction plan
contraction_plan = cutensor.create_contraction_plan(contract_desc, desc_A, desc_B, desc_output)

In [ ]:
#Perform contraction using contraction plan
cutensor.contraction(contraction_plan, alpha, A, desc_A, B, desc_B, beta, output, desc_output)

cp.cuda.Stream.null.synchronise()
print("Result with cutensor: ", output)

In [ ]:
#Task 1 function/abstract contraction passing the tensor, matrix, and dimension

def contract(A, X, dim):

    A_shape = A.shape
    X_shape = X.shape

    print(A_shape)

    A = A.reshape(A.prod(A.shape[:]))

    desc_A = cutensor.create_tensor_descriptor(A)
    desc_B = cutensor.create_tensor_descriptor(B)

    desc_output = cutensor.create_tensor_descriptor()
